In [83]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time
from random import randint


# start your browser

browser = webdriver.Chrome('./chromedriver')


browser.implicitly_wait(15)
wait = WebDriverWait(browser, 30)

# enter the URL you would like to scrape

urls = [
        #'file:///Users/chenchiungyi/Documents/Koopwoningen%20Amsterdam%20-%20Huizen%20te%20koop%20in%20Amsterdam%20[funda].htm',
    
        'http://www.funda.nl/koop/amsterdam/p'
    ]


data1 = []
data2 = []
all_data = []

# enter the pagerange you would like to scrape

pagerange = range(1, 2)

# start scraping

for url in urls:
    for page in pagerange:
        browser.get(url + str(page))
        # wait for the page to load
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.search-result-content")))

        for item in browser.find_elements_by_css_selector("div.search-result-content"):


            try:
                zipcode1, zipcode2, city = item.find_element_by_css_selector("h4.search-result__header-subtitle").text.split(" ", 2)
                zipcode = zipcode1 + " " + zipcode2

                street_zipcode_city = city
                #item.find_element_by_css_selector("h3.search-result-title").text

                price = item.find_element_by_css_selector("span.search-result-price").text.lstrip('€ ').rstrip(
                    ' k.k,').replace('.', '')

                surface, rooms = item.find_element_by_css_selector("ul.search-result-kenmerken").text.replace('\n', '').replace('m²', '').split(" ", 1)
                rooms = rooms.replace('kamer', '').replace('s', '')

                link = item.find_element_by_css_selector("div.search-result__header-title-col>a").get_attribute('href')

                data1.append({
                    "street_zipcode_city": street_zipcode_city,
                    "zipcode": zipcode,
                    "city": city,
                    "price": price,
                    "surface": surface,
                    "rooms": rooms,
                    "link": link,
                })

            except ValueError:
                pass

        time.sleep(randint(10, 15))

for j in range(len(data1)-7):
    
    browser.get(data1[j]['link'])
    
    for item in browser.find_elements_by_css_selector("dl.object-kenmerken-list"):


        try:
            temp_text = item.text.split()
            all_data.append(temp_text)
    
        except ValueError:
            pass
    
    
    
    if 'VvE' in all_data[0]:
        vve = all_data[0].index('VvE')
        vv_in = all_data[0][vve+1]
    else:
        vve_in = 0
    
    const_index = all_data[1].index('Bouwjaar')
    inh_index = all_data[2].index('Inhoud')
    
    if 'Aantal' in all_data[3]:
        aan_in = all_data[3].index('Aantal')
    
    if 'Gelgen' in all_data[3]:
        gel = all_data[3].index('Gelegen')
        gel_in = all_data[3][gel+2]
    else:
        gel_in = 'na'
    
    
    
    #organize data
    data2.append({
    "per_m2": all_data[0][10],
    "VvE": vve_in,
    "Type": all_data[1][1],
    "Const_yaer": all_data[1][const_index],
    "Volume": all_data[2][-2],
    "bedrooms": all_data[3][4],
    "located": all_data[3][aan_in+2],
    "floors": gel_in,
    "energy_lab": all_data[4][1]
                    
            })

time.sleep(randint(10, 15))

        
browser.close()
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
mergedDf = df1.merge(df2, left_index=True, right_index=True)
mergedDf.to_csv("AmsterdamPage"+str(min(pagerange))+"to"+str(max(pagerange))+".csv", sep=';', encoding='utf-8')
print(mergedDf)        



AttributeError: 'NoneType' object has no attribute 'is_displayed'

In [80]:
print(all_data[1])

['Soort', 'appartement', 'Bovenwoning', '(appartement)', 'Soort', 'bouw', 'Bestaande', 'bouw', 'Bouwjaar', '1993']


In [75]:
mergedDf = df2.merge(df2, left_index=True, right_index=True)
print(mergedDf.head())

  per_m2_x        VvE_x    Type_x  Const_yaer_x Volume_x bedrooms_x located_x  \
0        €  Aanvaarding  woonhuis  dakbedekking      505         (5        TV   
1        €  Aanvaarding  woonhuis  dakbedekking      505         (5        TV   
2        €  Aanvaarding  woonhuis  dakbedekking      505         (5        TV   
3        €  Aanvaarding  woonhuis  dakbedekking      505         (5        TV   
4        €  Aanvaarding  woonhuis  dakbedekking      505         (5        TV   

        floors_x energy_lab_x per_m2_y        VvE_y    Type_y  Const_yaer_y  \
0  Voorzieningen            E        €  Aanvaarding  woonhuis  dakbedekking   
1  Voorzieningen            E        €  Aanvaarding  woonhuis  dakbedekking   
2  Voorzieningen            E        €  Aanvaarding  woonhuis  dakbedekking   
3  Voorzieningen            E        €  Aanvaarding  woonhuis  dakbedekking   
4  Voorzieningen            E        €  Aanvaarding  woonhuis  dakbedekking   

  Volume_y bedrooms_y located_y       

In [67]:
#list(locate(all_data[1], lambda a: a == 'Bouwh=jaar'))
#con_index = all_data[1].index('Bouwjaar')
#print(all_data[1][con_index+1])
if 'VvE' in all_data[0]:
    vve_in = all_data[0].index('VvE')
    else:
    vve_in = 0

SyntaxError: invalid syntax (<ipython-input-67-e2d21e908aa4>, line 4)

In [73]:
print(df1)

Empty DataFrame
Columns: []
Index: []
